# Youtube 인기 급상승 채널로부터 크롤링 진행

In [48]:
# scraping에 필요한 라이브러리 호출
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore')

## 최신 기준 댓글 전부 긁어오기

In [52]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
url = "https://www.youtube.com/feed/trending"

options = webdriver.ChromeOptions()
# 크롬창 띄우기 X
# options.add_argument('headless')
options.add_argument('window-size=1920*1080')
# 크롤링 성능 향상
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR, en_US')
options.add_argument('user-agent=' + user_agent)

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = options)
driver.get(url)
driver.implicitly_wait(3)

# 카테고리별 클릭 버튼 추가하기
# ~~~ code

# 페이지 스크롤 다운 (모든 동영상 로딩)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(1)
    # 팝업 제거
    try:
        driver.find_element_by_css_selector('#dismiss-button > a').click()

    except:
        pass
    
# 모든 동영상 링크 수집

video_links = []
for video in driver.find_elements(By.XPATH, "//a[@id='video-title']"):
    video_links.append(video.get_attribute("href"))

In [53]:
comment_dict = dict()

for video_link in video_links:
    ## shorts는 스킵
    if 'shorts' in video_link:
        continue
    driver.get(video_link)
    driver.implicitly_wait(10)
    ## 총 댓글 갯수를 토대로 전체 댓글을 긁어올 때까지 반복
    comment_sum = int(driver.find_element(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[2]').text.replace(",", ''))
    comment_cnt = 0
    index = 0
    while comment_cnt < comment_sum:
        locator = (By.XPATH, '//*[@id="content-text"]')
        comment_elements = driver.find_elements(*locator)
        comment_cnt = len(comment_elements)
        
        #대댓글 숫자 파악해서 합산하면 정확하게 맞아떨어짐
        reply_elements = driver.find_elements(By.XPATH, '//*[@id="more-replies"]/yt-button-shape/button/div/span')
        
        for reply in reply_elements:
            reply_cnt = int(reply.text.replace("답글 ", "").replace("개", ""))
            comment_cnt += reply_cnt
            
        print("\r현재 수집 과정 : %s / %s" % (comment_cnt, comment_sum), end='', flush=True)
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:  # 페이지 맨아래로 스크롤 완료 상태
                break

            last_height = new_height
    comment_list = list()
    for c in comment_elements:
        comment_list.append(c.text)
    comment_dict[video_link] = comment_list
    print("  !!@@!! 완료 : %s" % video_link)

현재 수집 과정 : 15 / 15완료 : https://www.youtube.com/watch?v=H0VqCLbZupA
현재 수집 과정 : 2232 / 48487

KeyboardInterrupt: 

In [51]:
comment_dict

{'https://www.youtube.com/watch?v=H0VqCLbZupA': ['와우 의미를 부여하면서\n스토리를 풀어나가는 거\n넘 도움되네요\n심리에 대해서 배우고 가는 느낌 ',
  '마술의 흥미가있어',
  '요즘에 이거 보면서 마술에 흥미가 생겼어요♡',
  '형님 성공하셧네요',
  '만약 서로 다른색을 이야기하면 어떻게해야하나요\n처음에 빨강 그다음 검정 이런식으로',
  '첫번째거 어떻게\n하는지도 올려주새요',
  '구독하고갑니다',
  '세잎클로버나 스페이드를 선택하면.. 어떤식의 의미를 부여하는게 좋을까요.',
  '처음에 저게 마술인줄,,(1빠)',
  '10분 마술',
  '오늘 혀가\n많이 기네요\n잘배우고 갑니다 해햇',
  '2빠']}

# Comment Visualization